In [46]:
import pandas as pd
from pathlib import Path
import util

pd.set_option('display.float_format', '{:,.0%}'.format)


In [47]:


# list of equity geographies
equity_geogs = util.summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]

## Household VMT

In [48]:
# vmt data
df_vmt = pd.read_csv(util.output_path / 'agg/dash/person_vmt.csv')

# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

df_hh = pd.read_csv(util.output_path / 'agg/dash/hh_geog.csv')
df_hh['is_rgc'] = 'Not in RGC'
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

def vmt_per_hh(df_vmt, df_hh, geog, map=False):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

    df = _df_vmt.merge(df_hh, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
    
    
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Household'] = df['travdist_wt']/df['hhexpfac']

    
    return df[['Average Miles per Household']]


In [49]:
pd.set_option('display.float_format', '{:,.1f}'.format)
vmt_per_hh(df_vmt, df_hh, "hh_county")

,Average Miles per Household
hh_county,
King,26.6
Kitsap,31.0
Outside Region,79.0
Pierce,34.3
Snohomish,36.0
Region,30.3


In [50]:
vmt_per_hh(df_vmt, df_hh, "is_rgc")

,Average Miles per Household
is_rgc,
In RGC,9.3
Not in RGC,35.1
Region,30.3


In [51]:
vmt_per_hh(df_vmt, df_hh, "hh_rgc")

,Average Miles per Household
hh_rgc,
Auburn,18.1
Bellevue,5.2
Bothell Canyon Park,30.4
Bremerton,7.6
Burien,19.7
Everett,7.6
Federal Way,19.3
Greater Downtown Kirkland,19.2
Issaquah,19.5


In [52]:
pd.set_option('display.float_format', '{:,.1f}'.format)
vmt_per_hh(df_vmt, df_hh, "hh_rg_proposed")

,Average Miles per Household
hh_rg_proposed,
Cities and Towns,46.7
Core Cities,29.6
High Capacity Transit Communities,34.1
Metropolitan Cities,16.2
Rural Areas,58.6
Urban Unincorporated Areas,44.4
Region,30.3


In [53]:
efa_names = {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
}

df = pd.DataFrame()
for name, col in efa_names.items():
    df[name] = vmt_per_hh(df_vmt, df_hh, col, map=True)
df

,People of Color,Income,LEP,Disability,Older Adults,Youth
hh_efa_poc,,,,,,
Below Regional Average,34.7,33.9,31.8,32.1,28.3,24.0
Above Regional Average,25.9,28.7,28.9,31.2,33.2,37.6
Higher Share of Equity Population,25.7,21.6,27.1,24.1,32.0,43.4
Region,30.3,30.3,30.3,30.3,30.3,30.3


## Delay

In [54]:
df = pd.read_csv(util.output_path / 'agg/dash/trip_time_total.csv')
df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]

In [55]:
pd.options.display.float_format = '{:0,.1f}'.format
# Hours of delay for households in these locations
# df[['Total Delay Hours']]

def delay_per_hh(geog, map=False):

    df_hh = pd.read_csv(util.output_path / 'agg/dash/hh_geog.csv')
    df_hh['is_rgc'] = 'Not in RGC'
    df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

    df = pd.read_csv(util.output_path / 'agg/dash/trip_time_total.csv')
    df['is_rgc'] = 'Not in RGC'
    df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]
    df = df.groupby(geog).sum()[['travtime_wt']]

    df2 = pd.read_csv(util.output_path / 'agg/dash/trip_sov_ff_time.csv')
    df2['is_rgc'] = 'Not in RGC'
    df2.loc[df2['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df2 = df2[(df2['mode'].isin(['SOV','HOV2','HOV3+'])&(df2['dorp']==1))]
    df2 = df2.groupby(geog).sum()[['sov_ff_time_wt']]
    df = df2.merge(df, on=geog)

    # Hours of delay from travel time
    df['Total Delay Hours'] = (df['travtime_wt'] - df['sov_ff_time_wt'])/60
    # Set any negative delay to 0
    df.loc[df['Total Delay Hours'] < 0, 'Total Delay Hours'] = 0

    df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]
    
    
    # df.loc['Region',:] = df.sum(axis=0)
    # df['Average Miles per Household'] = df['travdist_wt']/df['hhexpfac']


    df = df.merge(df_hh, left_index=True, right_index=True)
    
    df['Average Minutes of Delay per HH'] = df['Total Delay Hours']/df['hhexpfac']*60

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    df['Annual Hours of Delay per HH'] = df['Average Minutes of Delay per HH']*util.summary_config['weekday_to_annual']/60

    df[['Total Delay Hours',
        'Annual Hours of Delay per HH']] = df[['Total Delay Hours', 'Annual Hours of Delay per HH']].astype(int).map('{:,}'.format)


    return df[['Total Delay Hours','Average Minutes of Delay per HH','Annual Hours of Delay per HH']]

df = delay_per_hh('hh_county')
df

,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_county,,,
King,"123,825",5.8,30
Kitsap,"8,025",3.2,17
Outside Region,0,7.2,38
Pierce,"62,382",7.5,39
Snohomish,"69,039",8.6,45


In [56]:
delay_per_hh('is_rgc')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
is_rgc,,,
In RGC,"13,838",1.9,9
Not in RGC,"249,434",7.6,40


In [57]:
delay_per_hh('hh_rgc')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_rgc,,,
Auburn,285,3.4,18
Bellevue,607,1.1,5
Bothell Canyon Park,894,10.1,53
Bremerton,63,0.7,3
Burien,655,4.5,23
Everett,469,1.3,6
Federal Way,382,3.7,19
Greater Downtown Kirkland,607,5.9,31
Issaquah,178,4.1,21


In [58]:
delay_per_hh('hh_rg_proposed')


,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
hh_rg_proposed,,,
Cities and Towns,"28,364",9.7,51
Core Cities,"68,309",7.3,38
High Capacity Transit Communities,"73,224",8.5,45
Metropolitan Cities,"43,865",3.1,16
Rural Areas,"33,965",8.9,47
Urban Unincorporated Areas,"15,543",11.2,59


In [59]:
df = pd.DataFrame()
for name, col in efa_names.items():
    _df = delay_per_hh(col, map=True)
    _df['Group'] = name
    df = pd.concat([df, _df])

df = df.reset_index()
df.rename(columns={'index':'EFA Type'}, inplace=True)

df[['Group', 'EFA Type', 'Total Delay Hours', 'Average Minutes of Delay per HH', 'Annual Hours of Delay per HH']]

,Group,EFA Type,Total Delay Hours,Average Minutes of Delay per HH,Annual Hours of Delay per HH
0,People of Color,Below Regional Average,"142,169",6.9,36
1,People of Color,Above Regional Average,"78,543",6.2,33
2,People of Color,Higher Share of Equity Population,"42,548",5.9,31
3,Income,Below Regional Average,"165,079",7.3,39
4,Income,Above Regional Average,"64,981",6.1,32
5,Income,Higher Share of Equity Population,"33,200",4.6,24
6,LEP,Below Regional Average,"151,410",6.2,33
7,LEP,Above Regional Average,"66,212",7.3,38
8,LEP,Higher Share of Equity Population,"45,639",6.6,34
9,Disability,Below Regional Average,"149,247",7.3,38


## Vehicle Ownership Distribution

percentage of households by number of vehicles available, by geography


In [60]:
pd.set_option('display.float_format', '{:,.0%}'.format)

hh = pd.read_csv(util.output_path / 'agg/dash/auto_ownership_efa.csv')
df_hh = hh.copy()
df_hh = df_hh[df_hh['hh_county']!="Outside Region"].copy()
# add home RGC
df_hh['is_rgc'] = 'Not in RGC'
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

equity_geogs = ['hh_efa_dis', 'hh_efa_older', 'hh_efa_lep', 'hh_efa_pov200', 'hh_efa_poc', 'hh_efa_youth']
df_hh[equity_geogs] = df_hh[equity_geogs].apply(lambda x: x.\
        map({0: 'Below Regional Average', 
             1: 'Above Regional Average', 
             2: 'Higher Share of Equity Population'}))

df_hh['hhvehs'] = df_hh['hhvehs'].map({0:"0 vehicle",
                                       1:"1 vehicle",
                                       2:"2 vehicles",
                                       3:"3 vehicles",
                                       4:"4 or more vehicles"})


In [61]:
def stat_by_geog(df, geog):
    """
    Function to calculate statistics by geography and vehicle ownership.
    """
    
    # Group by equity geography and vehicle ownership
    df_grouped = df.groupby([geog, 'hhvehs'], as_index=False)['hhexpfac'].sum()
    
    # Calculate total households in each equity geography
    total_hh = df.groupby([geog], as_index=False)['hhexpfac'].sum().rename(columns={'hhexpfac': 'total_hh'})
    
    # Merge the grouped data with total households
    df_merged = df_grouped.merge(total_hh, on=geog)
    
    # Calculate percentage of households with the specified vehicle ownership
    df_merged['percentage'] = df_merged['hhexpfac'] / df_merged['total_hh']
    
    return df_merged.pivot(index=geog, columns='hhvehs', values='percentage')


In [62]:
df_region = df_hh.groupby(['hhvehs'], as_index=False)['hhexpfac'].sum()
df_region['percentage'] = df_region['hhexpfac'] / df_hh['hhexpfac'].sum()
df_region['hh_county'] = 'Region'

In [63]:
df = stat_by_geog(df_hh, 'hh_county')
pd.concat([df_region.pivot(index='hh_county', columns='hhvehs', values='percentage'),df])


hhvehs,0 vehicle,1 vehicle,2 vehicles,3 vehicles,4 or more vehicles
hh_county,,,,,
Region,14%,35%,33%,12%,6%
King,17%,37%,30%,10%,5%
Kitsap,6%,30%,38%,17%,8%
Pierce,11%,33%,35%,14%,7%
Snohomish,9%,34%,36%,14%,8%


In [64]:
df = stat_by_geog(df_hh, 'hh_rg_proposed')
df

hhvehs,0 vehicle,1 vehicle,2 vehicles,3 vehicles,4 or more vehicles
hh_rg_proposed,,,,,
Cities and Towns,3%,28%,42%,17%,9%
Core Cities,11%,38%,33%,11%,6%
High Capacity Transit Communities,7%,34%,38%,14%,7%
Metropolitan Cities,26%,41%,24%,6%,3%
Rural Areas,1%,18%,43%,25%,13%
Urban Unincorporated Areas,3%,25%,44%,18%,10%


In [65]:
df = stat_by_geog(df_hh, 'is_rgc')
df

hhvehs,0 vehicle,1 vehicle,2 vehicles,3 vehicles,4 or more vehicles
is_rgc,,,,,
In RGC,41%,41%,14%,3%,1%
Not in RGC,7%,34%,37%,14%,7%


In [66]:
df = stat_by_geog(df_hh, 'hh_rgc')
df

hhvehs,0 vehicle,1 vehicle,2 vehicles,3 vehicles,4 or more vehicles
hh_rgc,,,,,
Auburn,28%,47%,18%,4%,3%
Bellevue,48%,38%,11%,2%,1%
Bothell Canyon Park,11%,43%,32%,9%,5%
Bremerton,34%,46%,15%,3%,2%
Burien,26%,46%,20%,5%,3%
Everett,38%,44%,14%,3%,1%
Federal Way,28%,44%,20%,5%,2%
Greater Downtown Kirkland,17%,53%,23%,5%,2%
Issaquah,22%,46%,25%,5%,2%


In [67]:
df = pd.DataFrame()
for name, col in efa_names.items():
    _df = stat_by_geog(df_hh,col).reset_index()
    _df['Group'] = name
    _df.rename(columns={col: 'EFA Type'}, inplace=True)
    df = pd.concat([df, _df])

df.set_index(['Group','EFA Type'])

hhvehs                                             0 vehicle  1 vehicle  \
Group           EFA Type                                                  
People of Color Above Regional Average                   18%        36%   
                Below Regional Average                   10%        33%   
                Higher Share of Equity Population        17%        39%   
Income          Above Regional Average                   14%        37%   
                Below Regional Average                   11%        33%   
                Higher Share of Equity Population        21%        41%   
LEP             Above Regional Average                   16%        36%   
                Below Regional Average                   13%        34%   
                Higher Share of Equity Population        14%        40%   
Disability      Above Regional Average                   12%        36%   
                Below Regional Average                   12%        34%   
                Higher Share of Equity Population        20%        38%   
Older Adults    Above Regional Average                   11%        34%   
                Below Regional Average                   16%        37%   
                Higher Share of Equity Population        12%        34%   
Youth           Above Regional Average                    6%        32%   
                Below Regional Average                   19%        38%   
                Higher Share of Equity Population         5%        29%   

hhvehs                                             2 vehicles  3 vehicles  \
Group           EFA Type                                                    
People of Color Above Regional Average                    30%         10%   
                Below Regional Average                    36%         14%   
                Higher Share of Equity Population         29%          9%   
Income          Above Regional Average                    32%         11%   
                Below Regional Average                    36%         14%   
                Higher Share of Equity Population         26%          8%   
LEP             Above Regional Average                    32%         11%   
                Below Regional Average                    34%         13%   
                Higher Share of Equity Population         31%         10%   
Disability      Above Regional Average                    33%         12%   
                Below Regional Average                    35%         13%   
                Higher Share of Equity Population         27%         10%   
Older Adults    Above Regional Average                    35%         14%   
                Below Regional Average                    31%         11%   
                Higher Share of Equity Population         34%         14%   
Youth           Above Regional Average                    38%         15%   
                Below Regional Average                    28%          9%   
                Higher Share of Equity Population         41%         17%   

hhvehs                                             4 or more vehicles  
Group           EFA Type                                               
People of Color Above Regional Average                             5%  
                Below Regional Average                             7%  
                Higher Share of Equity Population                  5%  
Income          Above Regional Average                             6%  
                Below Regional Average                             7%  
                Higher Share of Equity Population                  4%  
LEP             Above Regional Average                             6%  
                Below Regional Average                             7%  
                Higher Share of Equity Population                  5%  
Disability      Above Regional Average                             6%  
                Below Regional Average                             6%  
                Higher